In [1]:
import zipfile
import os
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import TensorBoard
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [2]:
#unzip the zip file state-farm-distracted-driver-detection.zip to its same directory located in ./content/drive/MyDrive/Datamining/

if os.path.exists('./content/drive/MyDrive/Datamining/imgs'):
    print("The zip file has already been extracted")
else:
    zip_file = './content/drive/MyDrive/Datamining/state-farm-distracted-driver-detection.zip'
    zip_ref = zipfile.ZipFile(zip_file, 'r')
    zip_ref.extractall('./content/drive/MyDrive/Datamining/')
    zip_ref.close()

The zip file has already been extracted


In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory('./content/drive/MyDrive/Datamining/imgs/train/', target_size=(224, 224), batch_size=32, class_mode='categorical', color_mode='rgb')
test_datagen = ImageDataGenerator(rescale=1./255) 
test_generator = test_datagen.flow_from_directory('./content/drive/MyDrive/Datamining/imgs/train/', target_size=(224, 224), batch_size=32, class_mode='categorical', color_mode='rgb')

Found 22424 images belonging to 10 classes.
<class 'keras.src.legacy.preprocessing.image.ImageDataGenerator'>
Found 22424 images belonging to 10 classes.


In [4]:
# Definition of the image sizes
input_size = 224
num_colors = 3

# Definition of the network layers
layers = [
    # First convolution with 16 filters of size 3 x 3 and relu activation function
    keras.layers.Conv2D(16,
                        (3,3),
                        activation='relu',
                        input_shape=(input_size,
                                    input_size,
                                    num_colors)),
    # Pooling of size 2 x 2 and stride 1
    keras.layers.MaxPooling2D(2, 2),
    # Second convolution with 32 filters of size 3 x 3 and relu activation function
    keras.layers.Conv2D(32,
                        (3,3),
                        activation='relu'),
    # Pooling of size 2 x 2 and stride 1
    keras.layers.MaxPooling2D(2,2),
    # Third convolution with 64 filters of size 3 x 3 and relu activation function
    keras.layers.Conv2D(64,
                        (3,3),
                        activation='relu'),
    # Pooling of size 2 x 2 and stride 1
    keras.layers.MaxPooling2D(2,2),
    # Fourth convolution with 64 filters of size 3 x 3 and relu activation function
    keras.layers.Conv2D(64,
                        (3,3),
                        activation='relu'),
    # Pooling of size 2 x 2 and stride 1
    keras.layers.MaxPooling2D(2,2),
    # Flattening to transform the information into a vector
    keras.layers.Flatten(),
    # Dense layer with 512 neurons and relu activation function
    keras.layers.Dense(512,
                       activation='relu'),
    # Output layer with softmax activation function
    keras.layers.Dense(10,
                       activation = tf.nn.softmax),
]

model = keras.Sequential(layers, name="myModel")

c:\Users\usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [5]:
# Optimization algorithm and loss function configuration
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Network structure
model.summary()

Model: "myModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,719,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,784,746 (18.25 MB)

 Trainable params: 4,784,746 (18.25 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:

# Training process execution
# Iterations: 15
# Number of steps per iteration: 4
history = model.fit(train_generator,
                    steps_per_epoch = 4,
                    epochs = 75)

# Evaluation of the model using the test set
results = model.evaluate(test_generator)

print(results)

Epoch 1/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 194ms/step - accuracy: 0.5156 - loss: 1.3893
Epoch 2/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 190ms/step - accuracy: 0.5885 - loss: 1.4323
Epoch 3/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.6562 - loss: 1.1260
Epoch 4/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.6844 - loss: 0.9934
Epoch 5/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 179ms/step - accuracy: 0.6729 - loss: 1.0816
Epoch 6/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step - accuracy: 0.6271 - loss: 1.0728
Epoch 7/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - accuracy: 0.6833 - loss: 0.9136
Epoch 8/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - accuracy: 0.8250 - loss: 0.5777
Epoch 9/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.7948 - loss: 0.6887
Epoch 10/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 181ms/step - accuracy: 0.7615 - loss: 0.8734
Epoch 11/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 179ms/step - accuracy: 0.7073 - loss: 0.7945
Epoch 12/75
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.8031 - lo